In [1]:
#データ整形
import json
import pprint

#データ読み込み
json_open_1 = open('C:/Users/owner/json_data/result_hashima.json', 'r', encoding="utf-8")
json_load_1 = json.load(json_open_1)
json_open_2 = open('C:/Users/owner/json_data/result_himeno.json', 'r', encoding="utf-8")
json_load_2 = json.load(json_open_2)
json_open_3 = open('C:/Users/owner/json_data/result_kitada.json', 'r', encoding="utf-8")
json_load_3 = json.load(json_open_3)
json_open_4 = open('C:/Users/owner/json_data/result_sugimori.json', 'r', encoding="utf-8")
json_load_4 = json.load(json_open_4)

#全データ
json_load = json_load_1 + json_load_2 + json_load_3 + json_load_4
print(len(json_load))
#処理後のデータ
id_list = []
data_list = []

#データ処理
for item in json_load:
    #データの型
    data = {
        "LO_ID": item["data"]["debate_id"],
        "pm_speech": item["data"]["pm_speech"],
        "lo_speech": item["data"]["lo_speech"],
         "score":
            {
                "justification": ["*", "*", "*", "*"], 
                "example":  ["*", "*", "*", "*"],
                "relevance":  ["*", "*", "*", "*"]
            },
         "component":
             {
            "component_text": [],
            "component_text_id": []
             }
    }




    #対象の論述構造
    for i in range(10):
        if item["data"]["eval_sent{}".format(i+1)] != "None":
            data["component"]["component_text"].append(item["data"]["eval_sent{}".format(i+1)])
            data["component"]["component_text_id"].append(i+1)

    num = id_list.count(item["data"]["debate_id"])
    
    #スコア
    if num == 0:
        for h in item["completions"][0]["result"]:
            if h["from_name"] == "eval_justification_score":
                data["score"]["justification"][0] = h["value"]["choices"][0]
            if h["from_name"] == "eval_example_score":
                data["score"]["example"][0] = h["value"]["choices"][0]
            if h["from_name"] == "eval_relevance_score":
                data["score"]["relevance"][0] = h["value"]["choices"][0]
        data_list.append(data)
                
    else:
        #debate_idが重複している要素のインデックス
        index = id_list.index(item["data"]["debate_id"])
        for h in item["completions"][0]["result"]:
            if h["from_name"] == "eval_justification_score":
                data_list[index]["score"]["justification"][num] = h["value"]["choices"][0]
            if h["from_name"] == "eval_example_score":
                data_list[index]["score"]["example"][num] = h["value"]["choices"][0]
            if h["from_name"] == "eval_relevance_score":
                data_list[index]["score"]["relevance"][num] = h["value"]["choices"][0]
        data_list.append("duplication")

    id_list.append(item["data"]["debate_id"])
    

print(len(data_list))
#余分なデータの削除
while "duplication" in data_list :
    data_list.remove("duplication")    
print(len(data_list))

#書き込み、保存
with open("debate.json", mode="w", encoding='utf-8') as f:
    d = json.dumps(data_list, indent=2, ensure_ascii=False)
    f.write(d)
    
with open('debate.json', 'r',  encoding='utf-8') as fp:
    data = json.load(fp)


629
629
315


In [13]:
#シンプルな一致率
import json
data_jus_A = []
data_jus_B = []
data_exa_A = []
data_exa_B = []
data_rel_A = []
data_rel_B = []

with open('debate.json', 'r',  encoding='utf-8') as fp:
    data = json.load(fp)

for item in data:
    distribution(item["score"]["justification"][0], data_jus_A)
    distribution(item["score"]["justification"][1], data_jus_B)
    distribution(item["score"]["example"][0], data_exa_A)
    distribution(item["score"]["example"][1], data_exa_B)
    distribution(item["score"]["relevance"][0], data_rel_A)
    distribution(item["score"]["relevance"][1], data_rel_B)

data_A = data_jus_A + data_exa_A + data_rel_A
data_B = data_jus_B + data_exa_B + data_rel_B

cor = 0
for i in range(len(data_A)):
    if data_A[i] == data_B[i]:
        cor = cor + 1

print(len(data_A))
print(cor)
print(float(cor/len(data_A)))

945
436
0.4613756613756614


In [2]:
#krippendorff's alpha
from __future__ import print_function
try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.


if __name__ == '__main__': 
    print("Example from http://en.wikipedia.org/wiki/Krippendorff's_Alpha")

    data = (
        "*    *    *    *    *    3    4    1    2    1    1    3    3    *    3", # coder A
        "1    *    2    1    3    3    4    3    *    *    *    *    *    *    *", # coder B
        "*    *    2    1    3    4    4    *    2    1    1    3    3    *    4", # coder C
    )

    missing = '*' # indicator for missing values
    array = [d.split() for d in data]  # convert to 2D list of string items
    print(array)
    print("nominal metric: %.3f" % krippendorff_alpha(array, nominal_metric, missing_items=missing))
    print("interval metric: %.3f" % krippendorff_alpha(array, interval_metric, missing_items=missing))

Example from http://en.wikipedia.org/wiki/Krippendorff's_Alpha
[['*', '*', '*', '*', '*', '3', '4', '1', '2', '1', '1', '3', '3', '*', '3'], ['1', '*', '2', '1', '3', '3', '4', '3', '*', '*', '*', '*', '*', '*', '*'], ['*', '*', '2', '1', '3', '4', '4', '*', '2', '1', '1', '3', '3', '*', '4']]
nominal metric: 0.691
interval metric: 0.811


In [3]:
#関数たち
def distribution(score, data_list):
    if score == "3-5点":
        data_list.append(4.0)
    elif score == "6点":
        data_list.append(6.0)
    elif score == "7-10点":
        data_list.append(8.5)
    elif score == "対象外":
        data_list.append("*")
    else:
        data_list.append("*")

#重み付きカッパーはfloatでやると回帰とごっちゃになってできない
def distribution_0(score, data_list):
    if score == "3-5点":
        data_list.append(4)
    elif score == "6点":
        data_list.append(6)
    elif score == "7-10点":
        data_list.append(8)
    elif score == "対象外":
        data_list.append(0)
    else:
        data_list.append(0)
        
def distribution_1(score, data_list):
    if score == "3-5点":
        data_list.append(1)
    elif score == "6点":
        data_list.append(2)
    elif score == "7-10点":
        data_list.append(3)
    elif score == "対象外":
        data_list.append("*")
    else:
        data_list.append("*")
        
def distribution_2(score, data_list):
    if score == "3-5点":
        data_list.append(1)
    elif score == "6点":
        data_list.append(2)
    elif score == "7-10点":
        data_list.append(3)
    elif score == "対象外":
        data_list.append(0)
    else:
        data_list.append(0)

In [4]:
#krippendorff's alpha (3-5点→4 としたやつ)
import json
data_jus_A = []
data_jus_B = []
data_exa_A = []
data_exa_B = []
data_rel_A = []
data_rel_B = []

with open('debate.json', 'r',  encoding='utf-8') as fp:
    data = json.load(fp)

for item in data:
    distribution(item["score"]["justification"][0], data_jus_A)
    distribution(item["score"]["justification"][1], data_jus_B)
    distribution(item["score"]["example"][0], data_exa_A)
    distribution(item["score"]["example"][1], data_exa_B)
    distribution(item["score"]["relevance"][0], data_rel_A)
    distribution(item["score"]["relevance"][1], data_rel_B)

data_A = data_jus_A + data_exa_A + data_rel_A
data_B = data_jus_B + data_exa_B + data_rel_B

data = []
data.append(data_A)
data.append(data_B)


missing = '*'
print(data)
print("nominal metric: %.3f" % krippendorff_alpha(data, nominal_metric, missing_items=missing))
print("interval metric: %.3f" % krippendorff_alpha(data, interval_metric, missing_items=missing))

[[6.0, 4.0, 4.0, 6.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 6.0, 6.0, 6.0, 6.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 6.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 6.0, 4.0, 6.0, 6.0, 4.0, 6.0, 4.0, 8.5, 4.0, 4.0, 6.0, 6.0, 4.0, 6.0, 6.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 6.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 6.0, 4.0, 8.5, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 8.5, 8.5, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 4.0, 4.0, 6.0, 6.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 6.0, 6.0, 4.0, 4.0, 6.0, 4.0, 8.5

nominal metric: 0.023
interval metric: 0.119


In [5]:
#krippendorff's alpha (3-5点→1 としたやつ)
import json
data_jus_A = []
data_jus_B = []
data_exa_A = []
data_exa_B = []
data_rel_A = []
data_rel_B = []

with open('debate.json', 'r',  encoding='utf-8') as fp:
    data = json.load(fp)

for item in data:
    distribution_1(item["score"]["justification"][0], data_jus_A)
    distribution_1(item["score"]["justification"][1], data_jus_B)
    distribution_1(item["score"]["example"][0], data_exa_A)
    distribution_1(item["score"]["example"][1], data_exa_B)
    distribution_1(item["score"]["relevance"][0], data_rel_A)
    distribution_1(item["score"]["relevance"][1], data_rel_B)

data_A = data_jus_A + data_exa_A + data_rel_A
data_B = data_jus_B + data_exa_B + data_rel_B

data = []
data.append(data_A)
data.append(data_B)


missing = '*'
print(data)
print("nominal metric: %.3f" % krippendorff_alpha(data, nominal_metric, missing_items=missing))
print("interval metric: %.3f" % krippendorff_alpha(data, interval_metric, missing_items=missing))

[[2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 3, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 3, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 3, 3, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 2, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 1, 1, 2, 1, 3, 2, 1, 1, 1, 2, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 3, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 3, 2, 2, 2, 1, 2, 2, '*', '*', 3, '*', '*', '*', '*', 1, '*', 2, '*', 2, '

In [46]:
#cohen's kappa(3-5点→4 としたやつ)
from sklearn.metrics import cohen_kappa_score
import json

data_jus_A = []
data_jus_B = []
data_exa_A = []
data_exa_B = []
data_rel_A = []
data_rel_B = []

with open('debate.json', 'r',  encoding='utf-8') as fp:
    data = json.load(fp)

for item in data:
    distribution_0(item["score"]["justification"][0], data_jus_A)
    distribution_0(item["score"]["justification"][1], data_jus_B)
    distribution_0(item["score"]["example"][0], data_exa_A)
    distribution_0(item["score"]["example"][1], data_exa_B)
    distribution_0(item["score"]["relevance"][0], data_rel_A)
    distribution_0(item["score"]["relevance"][1], data_rel_B)

data_A = data_jus_A + data_exa_A + data_rel_A
data_B = data_jus_B + data_exa_B + data_rel_B


cohen_kappa_score(data_A,data_B)

0.26308734086069274

In [8]:
#cohen's kappa(3-5点→1 としたやつ)
from sklearn.metrics import cohen_kappa_score
import json

data_jus_A = []
data_jus_B = []
data_exa_A = []
data_exa_B = []
data_rel_A = []
data_rel_B = []

with open('debate.json', 'r',  encoding='utf-8') as fp:
    data = json.load(fp)

for item in data:
    distribution_2(item["score"]["justification"][0], data_jus_A)
    distribution_2(item["score"]["justification"][1], data_jus_B)
    distribution_2(item["score"]["example"][0], data_exa_A)
    distribution_2(item["score"]["example"][1], data_exa_B)
    distribution_2(item["score"]["relevance"][0], data_rel_A)
    distribution_2(item["score"]["relevance"][1], data_rel_B)

data_A = data_jus_A + data_exa_A + data_rel_A
data_B = data_jus_B + data_exa_B + data_rel_B


cohen_kappa_score(data_A,data_B)

0.26308734086069274

In [ ]:
#学習用Jsonファイルの作成
import json
data_jus_A = []
data_jus_B = []
data_exa_A = []
data_exa_B = []
data_rel_A = []
data_rel_B = []

with open('debate.json', 'r',  encoding='utf-8') as fp:
    data = json.load(fp)

for item in data:
    distribution(item["score"]["justification"][0], data_jus_A)
    distribution(item["score"]["justification"][1], data_jus_B)
    distribution(item["score"]["example"][0], data_exa_A)
    distribution(item["score"]["example"][1], data_exa_B)
    distribution(item["score"]["relevance"][0], data_rel_A)
    distribution(item["score"]["relevance"][1], data_rel_B)

data_A = data_jus_A + data_exa_A + data_rel_A
data_B = data_jus_B + data_exa_B + data_rel_B

cor = 0

for i in range(len(data_A)):
    if data_A[i] == data_B[i]:
        